In [ ]:
import os
from dotenv import load_dotenv


load_dotenv()
API_KEY = os.getenv("GARMIN_API_KEY")
if not API_KEY:
    raise ValueError("Missing GARMIN_API_KEY environment variable")

https://medium.com/@shmilysyg/fastmcp-the-fastway-to-build-mcp-servers-aa14f88536d2

- uv pip install fastmcp



In [ ]:
# demo.py
# Test it with the MCP Inspector by fastmcp dev demo.py

from fastmcp import FastMCP

mcp = FastMCP("Demo")

# Add an addition tool
@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# Add a dynamic greeting resource
@mcp.resource("greeting://{name}")
def get_greeting(name: str) -> str:
    """Get a personalized greeting"""
    return f"Hello, {name}!"

In [ ]:
# echo.py
from mcp.server.fastmcp import FastMCP
from pydantic import Field

# Create server
mcp = FastMCP("Echo Server")


@mcp.tool()
def echo_tool(text: str = Field(description="The text to echo")) -> str:
    """Echo the input text

    Args:
        text (str): The text to echo

    Returns:
        str: The echoed text
    """
    return text

mcp.run()

## MCPAdapt 
https://grll.github.io/mcpadapt/guide/smolagents/
- pip install smolagents[mcp]
- pip install smolagents[mlx-lm]

how to interact with the server:

In [ ]:
from mcp import StdioServerParameters
from smolagents import CodeAgent, MLXModel
from smolagents.tools import ToolCollection

model = MLXModel(
    model_id="mlx-community/Qwen2.5-Coder-32B-Instruct-4bit",
    max_tokens=10000,
)

server_params = StdioServerParameters(command="uv", args=["run", "echo.py"])

with ToolCollection.from_mcp(server_params) as tools:
    agent = CodeAgent(tools=tools, model=model)
    agent.run("Can you echo something?")

[HF Agents](https://huggingface.co/docs/smolagents/en/guided_tour#agents---guided-tour)
- MLXModel creates a mlx-lm pipeline to run inference on your local machine.
-  [HfApiModel](https://huggingface.co/docs/smolagents/v1.13.0/en/reference/models#smolagents.HfApiModel)(model_id: str = 'Qwen/Qwen2.5-Coder-32B-Instruct')

## API

In [ ]:
from loguru import logger
import garth
from garth.exc import GarthException

SESSION_DIR = '.garth'

class Client:
    """
    A client class that provides methods to interact with Garmin Connect APIs.

    Attributes:
        _email: The Garmin account email address.
        _password: The Garmin account password.
    """

    def __init__(self, email: str, password: str):
        """
        Initializes the Client by attempting to log in to Garmin Connect.

        Args:
            email: Garmin account email address.
            password: Garmin account password.

        Raises:
            Exception: If login fails.
        """

        # Attempts to resume or perform a Garmin Connect login session, saving it for future operations.
        try:
            garth.resume(SESSION_DIR)
            # Ensure the session is loaded by accessing the username attribute.
            _ = garth.client.username
        except (FileNotFoundError, GarthException):
            garth.login(email, password)
            garth.save(SESSION_DIR)
    
        self._email = email
        self._password = password

        if not self.login():
            raise Exception("Login failed")
        

    def getAllWorkouts(self) -> dict:
        """
        Fetches all workouts from Garmin Connect.

        Returns:
            A list of dictionaries representing each workout.
        """
        return garth.connectapi(
            "/workout-service/workouts",
            params={
                "start": 1,
                "limit": 999,
                "myWorkoutsOnly": True,
                "sharedWorkoutsOnly": False,
                "orderBy": "WORKOUT_NAME",
                "orderSeq": "ASC",
                "includeAtp": False
            }
        )

    def deleteWorkout(self, workout: dict) -> bool:
        """
        Deletes a workout from Garmin Connect.

        Args:
            workout: A dictionary representing the workout to be deleted, which must contain 'workoutId' and 'workoutName'.

        Returns:
            True if the workout is deleted successfully, otherwise False.
        """
        res = garth.connectapi(
            f"/workout-service/workout/{workout['workoutId']}",
            method="DELETE"
        )
        if res is not None:
            logger.info(f"Deleted workoutId: {workout['workoutId']} workoutName: {workout['workoutName']}")
            return True
        else:
            logger.warning(
                f"Could not delete workout. Workout not found with workoutId: {workout['workoutId']} (workoutName: {workout['workoutName']})"
            )
            return False

    def scheduleWorkout(self, id: str, dateJson: dict) -> bool:
        """
        Schedules a workout on a specific date.

        Args:
            id: The workout ID to schedule.
            dateJson: A dictionary with a 'date' key in 'YYYY-MM-DD' format.

        Returns:
            True if scheduling is successful, otherwise False.
        """
        resJson = garth.connectapi(
            f"/workout-service/schedule/{id}",
            method="POST",
            headers={'Content-Type': 'application/json'},
            json=dateJson
        )
        if 'workoutScheduleId' not in resJson:
            return False
        return True

    def importWorkout(self, workoutJson: str) -> dict:
        """
        Imports a workout into Garmin Connect from a JSON string.

        Args:
            workoutJson: A JSON string representing a workout to import.

        Returns:
            A dictionary with the newly created workout's details (including 'workoutName').
        """
        resJson = garth.connectapi(
            "/workout-service/workout",
            method="POST",
            headers={'Content-Type': 'application/json'},
            data=workoutJson
        )
        logger.info(f"Imported workout {resJson['workoutName']}")
        return resJson


